In [1]:
import argparse
import os
import numpy as np
import math

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader, TensorDataset, Dataset
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch

In [2]:
x = np.load("data/xtrain32.npy")
y = np.load("data/ytrain.npy")
print(x[0])
print(x.shape)
print(y.shape)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(300000, 28, 56)
(300000,)


In [3]:
x = torch.Tensor(x)
y = torch.Tensor(y).to(torch.int8)

In [4]:

print(y[0])

tensor(5, dtype=torch.int8)


In [5]:
class CustomTensorDataset(Dataset):
    """TensorDataset with support of transforms.
    """
    def __init__(self, tensors, transform=None):
        assert all(tensors[0].size(0) == tensor.size(0) for tensor in tensors)
        self.tensors = tensors
        self.transform = transform

    def __getitem__(self, index):
        x = self.tensors[0][index]

        if self.transform:
            x = self.transform(x)

        y = self.tensors[1][index]

        return x, y

    def __len__(self):
        return self.tensors[0].size(0)

In [6]:
os.makedirs("images/50", exist_ok=True)


digit_embeddings = np.load("digit_embeddings.npy")

class Arguments():
  def __init__(self):
    self.n_epochs = 200
    self.batch_size = 64
    self.lr = 0.0002
    self.b1 = 0.5
    self.b2 = 0.999
    self.n_cpu = 8
    self.latent_dim = 100
    self.n_classes = 50
    self.img_size = 32
    self.channels = 1
    self.sample_interval = 1000
    self.embedding_size = 50

opt = Arguments()

img_shape = (opt.channels, opt.img_size, opt.img_size*2)

cuda = True if torch.cuda.is_available() else False


class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        self.label_emb = nn.Linear(opt.embedding_size, opt.embedding_size)

        def block(in_feat, out_feat, normalize=True):
            layers = [nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *block(opt.latent_dim + opt.embedding_size, 128, normalize=False),
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            nn.Linear(1024, int(np.prod(img_shape))),
            nn.Tanh()
        )

    def forward(self, noise, labels):
        # Concatenate label embedding and image to produce input
        gen_input = torch.cat((self.label_emb(labels), noise), -1)
        img = self.model(gen_input)
        img = img.view(img.size(0), *img_shape)
        return img


class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.label_embedding = nn.Linear(opt.embedding_size, opt.embedding_size)

        self.model = nn.Sequential(
            nn.Linear(opt.embedding_size + int(np.prod(img_shape)), 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 512),
            nn.Dropout(0.4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 512),
            nn.Dropout(0.4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 1),
        )

    def forward(self, img, labels):
        # Concatenate label embedding and image to produce input
        d_in = torch.cat((img.view(img.size(0), -1), self.label_embedding(labels)), -1)
        validity = self.model(d_in)
        return validity


# Loss functions
adversarial_loss = torch.nn.MSELoss()

# Initialize generator and discriminator
generator = Generator()
discriminator = Discriminator()

if cuda:
    generator.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()

# Configure data loader
dataloader = torch.utils.data.DataLoader(
    CustomTensorDataset(tensors = (x, y), transform = transforms.Compose(
            [transforms.ToPILImage(), transforms.Resize(opt.img_size), transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]
       )),
    batch_size=opt.batch_size,
    shuffle=True,
)

#

# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))

FloatTensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if cuda else torch.LongTensor

#numbers = np.random.randint(0, 49, [10])
#numbers = np.sort(numbers)
numbers = [1, 22, 23, 26, 29, 33, 35, 37, 42, 48]
print("Numbers", numbers)
def sample_image(n_row, batches_done):
    """Saves a grid of generated digits ranging from 0 to n_classes"""
    """does not use n_row"""
    # Sample noise
    z = Variable(FloatTensor(np.random.normal(0, 1, (10*10, opt.latent_dim))))
    # Get labels ranging from 0 to n_classes for n rows
    labels = np.array([num for _ in range(10) for num in numbers])
    gen_labels = Variable(FloatTensor(digit_embeddings[labels]))
    gen_imgs = generator(z, gen_labels)
    save_image(gen_imgs.data, "images/50/%d.png" % batches_done, nrow=10, normalize=True)


# ----------
#  Training
# ----------

for epoch in range(opt.n_epochs):
    for i, (imgs, labels) in enumerate(dataloader):

        batch_size = imgs.shape[0]

        # Adversarial ground truths
        valid = Variable(FloatTensor(batch_size, 1).fill_(1.0), requires_grad=False)
        fake = Variable(FloatTensor(batch_size, 1).fill_(0.0), requires_grad=False)

        # Configure input
        real_imgs = Variable(imgs.type(FloatTensor))
        labels = Variable(FloatTensor(digit_embeddings[labels]))

        # -----------------
        #  Train Generator
        # -----------------

        optimizer_G.zero_grad()

        # Sample noise and labels as generator input
        z = Variable(FloatTensor(np.random.normal(0, 1, (batch_size, opt.latent_dim))))
        gen_labels = Variable(FloatTensor(digit_embeddings[np.random.randint(0, opt.n_classes, batch_size)]))
        # Generate a batch of images
        gen_imgs = generator(z, gen_labels)
        
        # Loss measures generator's ability to fool the discriminator
        validity = discriminator(gen_imgs, gen_labels)
        g_loss = adversarial_loss(validity, valid)

        g_loss.backward()
        optimizer_G.step()

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()

        # Loss for real images
        validity_real = discriminator(real_imgs, labels)
        d_real_loss = adversarial_loss(validity_real, valid)

        # Loss for fake images
        validity_fake = discriminator(gen_imgs.detach(), gen_labels)
        d_fake_loss = adversarial_loss(validity_fake, fake)

        # Total discriminator loss
        d_loss = (d_real_loss + d_fake_loss) / 2

        d_loss.backward()
        optimizer_D.step()
        if i%1000==0:
            print(
                "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
                % (epoch, opt.n_epochs, i, len(dataloader), d_loss.item(), g_loss.item())
            )

        batches_done = epoch * len(dataloader) + i
        if batches_done % opt.sample_interval == 0:
            sample_image(n_row=20, batches_done=batches_done)

Numbers [1, 22, 23, 26, 29, 33, 35, 37, 42, 48]
[Epoch 0/200] [Batch 0/4688] [D loss: 0.457435] [G loss: 0.962952]
[Epoch 0/200] [Batch 1000/4688] [D loss: 0.246377] [G loss: 0.249058]
[Epoch 0/200] [Batch 2000/4688] [D loss: 0.248893] [G loss: 0.244712]
[Epoch 0/200] [Batch 3000/4688] [D loss: 0.244984] [G loss: 0.273848]
[Epoch 0/200] [Batch 4000/4688] [D loss: 0.244644] [G loss: 0.264036]
[Epoch 1/200] [Batch 0/4688] [D loss: 0.252201] [G loss: 0.253050]
[Epoch 1/200] [Batch 1000/4688] [D loss: 0.251820] [G loss: 0.255851]
[Epoch 1/200] [Batch 2000/4688] [D loss: 0.248889] [G loss: 0.279270]
[Epoch 1/200] [Batch 3000/4688] [D loss: 0.246908] [G loss: 0.215853]
[Epoch 1/200] [Batch 4000/4688] [D loss: 0.251058] [G loss: 0.255259]
[Epoch 2/200] [Batch 0/4688] [D loss: 0.249478] [G loss: 0.250518]
[Epoch 2/200] [Batch 1000/4688] [D loss: 0.266143] [G loss: 0.245598]
[Epoch 2/200] [Batch 2000/4688] [D loss: 0.242533] [G loss: 0.258739]
[Epoch 2/200] [Batch 3000/4688] [D loss: 0.243123] 

[Epoch 23/200] [Batch 1000/4688] [D loss: 0.215652] [G loss: 0.391505]
[Epoch 23/200] [Batch 2000/4688] [D loss: 0.222133] [G loss: 0.301776]
[Epoch 23/200] [Batch 3000/4688] [D loss: 0.214047] [G loss: 0.259669]
[Epoch 23/200] [Batch 4000/4688] [D loss: 0.240520] [G loss: 0.148434]
[Epoch 24/200] [Batch 0/4688] [D loss: 0.236007] [G loss: 0.296397]
[Epoch 24/200] [Batch 1000/4688] [D loss: 0.211243] [G loss: 0.350189]
[Epoch 24/200] [Batch 2000/4688] [D loss: 0.219551] [G loss: 0.258571]
[Epoch 24/200] [Batch 3000/4688] [D loss: 0.206185] [G loss: 0.375058]
[Epoch 24/200] [Batch 4000/4688] [D loss: 0.293883] [G loss: 0.503525]
[Epoch 25/200] [Batch 0/4688] [D loss: 0.216717] [G loss: 0.335201]
[Epoch 25/200] [Batch 1000/4688] [D loss: 0.201251] [G loss: 0.375304]
[Epoch 25/200] [Batch 2000/4688] [D loss: 0.195525] [G loss: 0.372774]
[Epoch 25/200] [Batch 3000/4688] [D loss: 0.204135] [G loss: 0.566992]
[Epoch 25/200] [Batch 4000/4688] [D loss: 0.239711] [G loss: 0.261751]
[Epoch 26/20

[Epoch 46/200] [Batch 2000/4688] [D loss: 0.176067] [G loss: 0.473062]
[Epoch 46/200] [Batch 3000/4688] [D loss: 0.179059] [G loss: 0.438571]
[Epoch 46/200] [Batch 4000/4688] [D loss: 0.195046] [G loss: 0.369775]
[Epoch 47/200] [Batch 0/4688] [D loss: 0.154892] [G loss: 0.542294]
[Epoch 47/200] [Batch 1000/4688] [D loss: 0.150064] [G loss: 0.598671]
[Epoch 47/200] [Batch 2000/4688] [D loss: 0.167604] [G loss: 0.382281]
[Epoch 47/200] [Batch 3000/4688] [D loss: 0.162782] [G loss: 0.425661]
[Epoch 47/200] [Batch 4000/4688] [D loss: 0.172079] [G loss: 0.420341]
[Epoch 48/200] [Batch 0/4688] [D loss: 0.168966] [G loss: 0.499750]
[Epoch 48/200] [Batch 1000/4688] [D loss: 0.166946] [G loss: 0.601422]
[Epoch 48/200] [Batch 2000/4688] [D loss: 0.164195] [G loss: 0.506110]
[Epoch 48/200] [Batch 3000/4688] [D loss: 0.140451] [G loss: 0.527338]
[Epoch 48/200] [Batch 4000/4688] [D loss: 0.171695] [G loss: 0.391536]
[Epoch 49/200] [Batch 0/4688] [D loss: 0.183259] [G loss: 0.560354]
[Epoch 49/200] 

[Epoch 69/200] [Batch 3000/4688] [D loss: 0.134705] [G loss: 0.474614]
[Epoch 69/200] [Batch 4000/4688] [D loss: 0.192033] [G loss: 0.571949]
[Epoch 70/200] [Batch 0/4688] [D loss: 0.165886] [G loss: 0.490886]
[Epoch 70/200] [Batch 1000/4688] [D loss: 0.155628] [G loss: 0.394113]
[Epoch 70/200] [Batch 2000/4688] [D loss: 0.176813] [G loss: 0.589015]
[Epoch 70/200] [Batch 3000/4688] [D loss: 0.122763] [G loss: 0.542323]
[Epoch 70/200] [Batch 4000/4688] [D loss: 0.167341] [G loss: 0.402179]
[Epoch 71/200] [Batch 0/4688] [D loss: 0.157340] [G loss: 0.524566]
[Epoch 71/200] [Batch 1000/4688] [D loss: 0.156193] [G loss: 0.646850]
[Epoch 71/200] [Batch 2000/4688] [D loss: 0.196558] [G loss: 0.329974]
[Epoch 71/200] [Batch 3000/4688] [D loss: 0.177245] [G loss: 0.523431]
[Epoch 71/200] [Batch 4000/4688] [D loss: 0.137829] [G loss: 0.585688]
[Epoch 72/200] [Batch 0/4688] [D loss: 0.181047] [G loss: 0.363899]
[Epoch 72/200] [Batch 1000/4688] [D loss: 0.161000] [G loss: 0.565419]
[Epoch 72/200] 

[Epoch 92/200] [Batch 4000/4688] [D loss: 0.178802] [G loss: 0.623074]
[Epoch 93/200] [Batch 0/4688] [D loss: 0.150539] [G loss: 0.594306]
[Epoch 93/200] [Batch 1000/4688] [D loss: 0.114374] [G loss: 0.687733]
[Epoch 93/200] [Batch 2000/4688] [D loss: 0.148666] [G loss: 0.635372]
[Epoch 93/200] [Batch 3000/4688] [D loss: 0.130079] [G loss: 0.507533]
[Epoch 93/200] [Batch 4000/4688] [D loss: 0.156543] [G loss: 0.388943]
[Epoch 94/200] [Batch 0/4688] [D loss: 0.136982] [G loss: 0.582811]
[Epoch 94/200] [Batch 1000/4688] [D loss: 0.138918] [G loss: 0.625504]
[Epoch 94/200] [Batch 2000/4688] [D loss: 0.110242] [G loss: 0.806286]
[Epoch 94/200] [Batch 3000/4688] [D loss: 0.146742] [G loss: 0.548056]
[Epoch 94/200] [Batch 4000/4688] [D loss: 0.160412] [G loss: 0.402836]
[Epoch 95/200] [Batch 0/4688] [D loss: 0.152200] [G loss: 0.528581]
[Epoch 95/200] [Batch 1000/4688] [D loss: 0.153916] [G loss: 0.523834]
[Epoch 95/200] [Batch 2000/4688] [D loss: 0.112046] [G loss: 0.669241]
[Epoch 95/200] 

[Epoch 115/200] [Batch 4000/4688] [D loss: 0.157094] [G loss: 0.662823]
[Epoch 116/200] [Batch 0/4688] [D loss: 0.106427] [G loss: 0.749799]
[Epoch 116/200] [Batch 1000/4688] [D loss: 0.129981] [G loss: 0.565241]
[Epoch 116/200] [Batch 2000/4688] [D loss: 0.101730] [G loss: 0.801313]
[Epoch 116/200] [Batch 3000/4688] [D loss: 0.113048] [G loss: 0.668790]
[Epoch 116/200] [Batch 4000/4688] [D loss: 0.119651] [G loss: 0.694148]
[Epoch 117/200] [Batch 0/4688] [D loss: 0.149573] [G loss: 0.586764]
[Epoch 117/200] [Batch 1000/4688] [D loss: 0.121795] [G loss: 0.592493]
[Epoch 117/200] [Batch 2000/4688] [D loss: 0.112732] [G loss: 0.681178]
[Epoch 117/200] [Batch 3000/4688] [D loss: 0.111261] [G loss: 0.492649]
[Epoch 117/200] [Batch 4000/4688] [D loss: 0.121244] [G loss: 0.642495]
[Epoch 118/200] [Batch 0/4688] [D loss: 0.112934] [G loss: 0.515984]
[Epoch 118/200] [Batch 1000/4688] [D loss: 0.115310] [G loss: 0.636364]
[Epoch 118/200] [Batch 2000/4688] [D loss: 0.113848] [G loss: 0.724360]
[

[Epoch 138/200] [Batch 3000/4688] [D loss: 0.102471] [G loss: 0.630278]
[Epoch 138/200] [Batch 4000/4688] [D loss: 0.104251] [G loss: 0.580227]
[Epoch 139/200] [Batch 0/4688] [D loss: 0.169003] [G loss: 0.381224]
[Epoch 139/200] [Batch 1000/4688] [D loss: 0.098583] [G loss: 0.706650]
[Epoch 139/200] [Batch 2000/4688] [D loss: 0.102139] [G loss: 0.702117]
[Epoch 139/200] [Batch 3000/4688] [D loss: 0.123758] [G loss: 0.664964]
[Epoch 139/200] [Batch 4000/4688] [D loss: 0.079095] [G loss: 0.810292]
[Epoch 140/200] [Batch 0/4688] [D loss: 0.082351] [G loss: 0.717705]
[Epoch 140/200] [Batch 1000/4688] [D loss: 0.097082] [G loss: 0.568859]
[Epoch 140/200] [Batch 2000/4688] [D loss: 0.094635] [G loss: 0.629803]
[Epoch 140/200] [Batch 3000/4688] [D loss: 0.094916] [G loss: 0.740366]
[Epoch 140/200] [Batch 4000/4688] [D loss: 0.092689] [G loss: 0.632232]
[Epoch 141/200] [Batch 0/4688] [D loss: 0.107047] [G loss: 0.555655]
[Epoch 141/200] [Batch 1000/4688] [D loss: 0.084984] [G loss: 0.708300]
[

[Epoch 161/200] [Batch 2000/4688] [D loss: 0.085809] [G loss: 0.644749]
[Epoch 161/200] [Batch 3000/4688] [D loss: 0.074438] [G loss: 0.708595]
[Epoch 161/200] [Batch 4000/4688] [D loss: 0.095023] [G loss: 0.880102]
[Epoch 162/200] [Batch 0/4688] [D loss: 0.071847] [G loss: 0.894380]
[Epoch 162/200] [Batch 1000/4688] [D loss: 0.093451] [G loss: 0.878483]
[Epoch 162/200] [Batch 2000/4688] [D loss: 0.081106] [G loss: 0.645998]
[Epoch 162/200] [Batch 3000/4688] [D loss: 0.084824] [G loss: 0.849919]
[Epoch 162/200] [Batch 4000/4688] [D loss: 0.081470] [G loss: 0.783135]
[Epoch 163/200] [Batch 0/4688] [D loss: 0.086329] [G loss: 0.843010]
[Epoch 163/200] [Batch 1000/4688] [D loss: 0.086074] [G loss: 0.692192]
[Epoch 163/200] [Batch 2000/4688] [D loss: 0.082936] [G loss: 0.747461]
[Epoch 163/200] [Batch 3000/4688] [D loss: 0.123575] [G loss: 0.950116]
[Epoch 163/200] [Batch 4000/4688] [D loss: 0.094227] [G loss: 0.715355]
[Epoch 164/200] [Batch 0/4688] [D loss: 0.085429] [G loss: 0.711391]
[

KeyboardInterrupt: 